In [17]:
import pandas as pd
import email
import os
import 

In [4]:
data_path = "Data/Raw_data"
chinese_path = "trec06c"
english_path = "trec06p"
index_path = "full/index"

In [22]:
labels_ch = pd.read_csv(os.path.join(data_path,chinese_path,index_path), sep=" ", header=None)
labels_en = pd.read_csv(os.path.join(data_path,english_path,index_path), sep=" ", header=None)
labels_ch.columns = ["class", "subpath"]
labels_en.columns = ["class", "subpath"]

In [33]:
def extract_message(row, lang):
    subpath = row["subpath"][3:]
    if lang=="en":
        full_path = os.path.join(data_path, english_path, subpath)
        encode_type = 'ISO-8859-1'
    elif lang=="ch":
        full_path = os.path.join(data_path, chinese_path, subpath)
        encode_type = 'GB2312'
    try:
        with open(full_path, "r", encoding=encode_type) as f:
            email_content = f.read()
            parsed_email_content = email.message_from_string(email_content)
            row["subject"] = str(email.header.make_header(email.header.decode_header(parsed_email_content["subject"])))
            row["email_to"] = str(email.header.make_header(email.header.decode_header(parsed_email_content["to"])))
            row["email_from"] = str(email.header.make_header(email.header.decode_header(parsed_email_content["from"])))
    except:
        row["subject"] = None
        row["email_to"] = None
        row["email_from"] = None
        row["message"] = None
        return row
    if parsed_email_content.is_multipart():
        row["message"] = list(parsed_email_content.get_payload())
    else:
        row["message"] = [parsed_email_content.get_payload()]
    return row

extract_en = lambda row: extract_message(row, lang="en")
extract_ch = lambda row: extract_message(row, lang="ch")

In [34]:
ch_emails = labels_ch.apply(extract_ch, axis=1)

In [35]:
ch_emails.dropna(inplace=True)

In [36]:
ch_emails.head()

,class,subpath,subject,email_to,email_from,message
1,ham,../data/000/001,● 问一部魏宗万的电影名称,shi@ccert.edu.cn,"""pan"" <pan@jdl.ac.cn>",[讲的是孔子后人的故事。一个老领导回到家乡，跟儿子感情不和，跟贪财的孙子孔为本和睦。\n老领...
2,spam,../data/000/002,公司业务.代开发票！,xing@ccert.edu.cn,张海南 <jian@163.con>,[尊敬的贵公司(财务/经理)负责人您好！ \n 我是深圳金海实业有限公司（广...
3,spam,../data/000/003,低点代开发票!,ling@ccert.edu.cn,代开发票 <pan@12.com>,[贵公司负责人(经理/财务）您好： \n 深圳市华龙公司受多家公司委托向外低点代开部分...
4,spam,../data/000/004,一边上网冲浪，一边赚钱，何乐而不为？,tang@ccert.edu.cn,"""mei"" <mei@dghhkjk.com>",[这是一封HTML格式信件！\n\n----------------------------...
5,spam,../data/000/005,优惠代开各种发票!!,yuan@ccert.edu.cn,"ke@163.com"" <chunyang-sz@163.com>",[\nTO：贵公司经理、财务\n\n \n 您好！ \n 深圳市春洋贸易...


In [45]:
ch_emails.to_pickle("Data/ch_emails_raw.pkl")

In [39]:
en_emails = labels_en.apply(extract_en, axis=1)

In [40]:
en_emails.dropna(inplace=True)

In [42]:
len(en_emails)

36837

In [44]:
en_emails.to_pickle("Data/en_emails_raw.pkl")

In [46]:
pd.read_pickle("Data/ch_emails_raw.pkl")

,class,subpath,subject,email_to,email_from,message
1,ham,../data/000/001,● 问一部魏宗万的电影名称,shi@ccert.edu.cn,"""pan"" <pan@jdl.ac.cn>",[讲的是孔子后人的故事。一个老领导回到家乡，跟儿子感情不和，跟贪财的孙子孔为本和睦。\n老领...
2,spam,../data/000/002,公司业务.代开发票！,xing@ccert.edu.cn,张海南 <jian@163.con>,[尊敬的贵公司(财务/经理)负责人您好！ \n 我是深圳金海实业有限公司（广...
3,spam,../data/000/003,低点代开发票!,ling@ccert.edu.cn,代开发票 <pan@12.com>,[贵公司负责人(经理/财务）您好： \n 深圳市华龙公司受多家公司委托向外低点代开部分...
4,spam,../data/000/004,一边上网冲浪，一边赚钱，何乐而不为？,tang@ccert.edu.cn,"""mei"" <mei@dghhkjk.com>",[这是一封HTML格式信件！\n\n----------------------------...
5,spam,../data/000/005,优惠代开各种发票!!,yuan@ccert.edu.cn,"ke@163.com"" <chunyang-sz@163.com>",[\nTO：贵公司经理、财务\n\n \n 您好！ \n 深圳市春洋贸易...
...,...,...,...,...,...,...
64615,spam,../data/215/115,优惠代开发票~!!!,guo@ccert.edu.cn,刘伟源 <ning@163.com>,[贵公司负责人(经理/财务)您好：\n 我公司是深圳市华源实业有限公司.本公司实力雄厚...
64616,spam,../data/215/116,代开发票验证后付款！,mo@ccert.edu.cn,优惠代开发票 <zhong@12.com>,[尊敬的商家朋友您好： \n 我是深圳市裕华实业有限公司的。我司实力雄厚，有着良\n好...
64617,spam,../data/215/117,优惠代开各类发票！,zeng@ccert.edu.cn,杨先生 <che@163.com>,[贵公司负责人(经理/财务）您好! \n 我...
64618,spam,../data/215/118,千色坊―时尚女士箱包特卖场,long@ccert.edu.cn,zeng@126.com,[\n 这是一个HTML格式的邮件\n FRAME: easymain\n\n\n\n]
